In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from eeg_net.eeg_cnn import * 
from eeg_net.eeg_rnn import *
import numpy as np 
import matplotlib.pyplot as plt 
from torchsummary import summary
%load_ext autoreload
%autoreload 2
%matplotlib inline
#plt.rcParams['figure.figsize'] = [12, 8]
#plt.rcParams['figure.dpi'] = 200
X_test = np.load("data/X_test.npy")
y_test = np.load("data/y_test.npy")
person_train_valid = np.load("data/person_train_valid.npy")
X_train_valid = np.load("data/X_train_valid.npy")
y_train_valid = np.load("data/y_train_valid.npy")
person_test = np.load("data/person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
X_test_dir = './data/X_test.npy'
y_test_dir = './data/y_test.npy' 
X_train_valid_dir = './data/X_train_valid.npy' 
y_train_valid_dir = './data/y_train_valid.npy'
X_test_dsample_dir = './data/X_test_downsample.npy'
y_test_dsample_dir = './data/y_test_downsample.npy' 
X_train_valid_dsample_dir = './data/X_train_valid_downsample.npy' 
y_train_valid_dsample_dir = './data/y_train_valid_downsample.npy'
X_test_ds = np.load(X_test_dsample_dir)
y_test_ds = np.load(y_test_dsample_dir)
X_train_valid_ds = np.load(X_train_valid_dsample_dir)
y_train_valid_ds = np.load(y_train_valid_dsample_dir)
print ('Training/Valid data shape: {}'.format(X_train_valid_ds.shape))
print ('Test data shape: {}'.format(X_test_ds.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid_ds.shape))
print ('Test target shape: {}'.format(y_test_ds.shape))
torch.manual_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
Training/Valid data shape: (4230, 22, 500)
Test data shape: (886, 22, 500)
Training/Valid target shape: (4230,)
Test target shape: (886,)


In [6]:
rnn = nn.LSTM(input_size=10, hidden_size=20, num_layers=2,batch_first=True)#(input_size,hidden_size,num_layers)
input = torch.randn(3, 5, 10)#(seq_len, batch, input_size)
h0 = torch.randn(2, 3, 20) #(num_layers,batch,output_size)
c0 = torch.randn(2, 3, 20) #(num_layers,batch,output_size)
output, (hn, cn) = rnn(input, (h0, c0))
print(output.shape)


torch.Size([3, 5, 20])


In [26]:
input_size = 22
hidden_size = 60
num_layers = 1 
num_classes = 4 
options = {
    'train_batch_size': 32,
    'scheduler_patience': 100,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 1e-4,
    'epoch_num': 200,
    'transform': 'none',
    #'downsample_split': True,
    'save_flag':False,
    'model_name': 'native_lstm.pth', 
    'min_save_epoch': 50, 

}
model = NativeEEGLSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        num_classes=num_classes).to('cuda')

loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dir,
        label_dir=y_train_valid_dir,
        preload_gpu=True)

Start training...
Epoch	Train Loss	Train Acc	Test Loss	Test_Acc	
--Iter 49	1.392355
0	1.387312	0.244693	1.386227	0.262411	
--Iter 49	1.392489
1	1.386879	0.248652	1.386314	0.267139	
--Iter 49	1.371034
2	1.386439	0.260697	1.386461	0.264775	
--Iter 49	1.376724


KeyboardInterrupt: 

In [30]:
input_size = 22
hidden_size = 128
num_layers = 1 
num_classes = 4 
options = {
    'train_batch_size': 32,
    'scheduler_patience': 100,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 1e-5,
    'epoch_num': 200,
    'transform': 'none',
    #'downsample_split': True,
    'save_flag':False,
    'model_name': 'native_lstm.pth', 
    'min_save_epoch': 50, 

}
model = NativeEEGLSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        num_classes=num_classes).to('cuda')

loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dsample_dir,
        label_dir=y_train_valid_dsample_dir,
        preload_gpu=True)

Start training...
Epoch	Train Loss	Train Acc	Test Loss	Test_Acc	
--Iter 49	1.397986
--Iter 99	1.400007
0	1.386397	0.260810	1.388076	0.258865	
--Iter 49	1.395096
--Iter 99	1.396359
1	1.386243	0.258943	1.387988	0.261229	
--Iter 49	1.388370
--Iter 99	1.384054
2	1.386142	0.258943	1.387919	0.263593	
--Iter 49	1.380115
--Iter 99	1.392395
3	1.386025	0.261498	1.387855	0.264775	
--Iter 49	1.386133
--Iter 99	1.374818
4	1.385919	0.259827	1.387791	0.267139	
--Iter 49	1.381653
--Iter 99	1.372764
5	1.385842	0.262677	1.387716	0.258865	
--Iter 49	1.385693
--Iter 99	1.389923
6	1.385757	0.260122	1.387677	0.257683	
--Iter 49	1.387294
--Iter 99	1.381955
7	1.385588	0.260613	1.387572	0.254137	
--Iter 49	1.382916
--Iter 99	1.375725
8	1.385507	0.263463	1.387544	0.250591	
--Iter 49	1.393958
--Iter 99	1.379509
9	1.385399	0.262382	1.387471	0.252955	
--Iter 49	1.386296
--Iter 99	1.396606
10	1.385364	0.263758	1.387437	0.251773	
--Iter 49	1.387173
--Iter 99	1.380032
11	1.385268	0.266804	1.387394	0.250591	
--Iter 49

KeyboardInterrupt: 

In [65]:
input_size = 22
hidden_size = 60
num_layers = 1 
num_classes = 4 
options = {
    'train_batch_size': 64,
    'scheduler_patience': 100,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 2e-5,
    'epoch_num': 200,
    'transform': 'none',
    #'downsample_split': True,
    'save_flag':False,
    'model_name': 'native_lstm.pth', 
    'min_save_epoch': 50, 
}
model_options= {}
model = CNNEEGLSTM(in_channels=22,options=model_options).to('cuda')
loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dsample_dir,
        label_dir=y_train_valid_dsample_dir,
        preload_gpu=True)

Start training...
Epoch	Train Loss	Train Acc	Test Loss	Test_Acc	
--Iter 49	1.384905
0	1.386250	0.258044	1.386719	0.228132	
--Iter 49	1.387752
1	1.386249	0.257412	1.386720	0.231678	
--Iter 49	1.386760
2	1.386256	0.254928	1.386726	0.235225	
--Iter 49	1.385814
3	1.386249	0.256612	1.386732	0.239953	
--Iter 49	1.386572
4	1.386247	0.256023	1.386735	0.239953	
--Iter 49	1.385525
5	1.386249	0.255349	1.386744	0.239953	
--Iter 49	1.386211
6	1.386246	0.253496	1.386739	0.237589	
--Iter 49	1.386930
7	1.386247	0.254717	1.386747	0.235225	
--Iter 49	1.386439
8	1.386248	0.254801	1.386745	0.236407	
--Iter 49	1.385891
9	1.386248	0.254591	1.386752	0.234043	
--Iter 49	1.385256
10	1.386246	0.256360	1.386751	0.232861	
--Iter 49	1.386787
11	1.386253	0.255264	1.386748	0.231678	
--Iter 49	1.387038
12	1.386250	0.256233	1.386746	0.229314	
--Iter 49	1.386992
13	1.386256	0.256023	1.386747	0.228132	
--Iter 49	1.386650
14	1.386251	0.256065	1.386744	0.229314	
--Iter 49	1.385734
15	1.386256	0.256612	1.386741	0.228132	
-

KeyboardInterrupt: 